In [1]:
from sympy import symbols, sin, cos, pi, pprint, simplify, atan2, sqrt, atan
from sympy.matrices import Matrix, Transpose
from IPython.display import display
import ipywidgets as widgets
from functools import reduce
import numpy as np
from importlib import reload

In [88]:
import kuka_arm.scripts.inverse_kinematics as ik
reload(ik)

test_cases = {1:[[[2.16135,-1.42635,1.55109],
                  [0.708611,0.186356,-0.157931,0.661967]],
                  [1.89451,-1.44302,1.69366],
                  [-0.65,0.45,-0.36,0.95,0.79,0.49]],
              2:[[[-0.56754,0.93663,3.0038],
                  [0.62073, 0.48318,0.38759,0.480629]],
                  [-0.638,0.64198,2.9988],
                  [-0.79,-0.11,-2.33,1.94,1.14,-3.68]],
              3:[[[-1.3863,0.02074,0.90986],
                  [0.01735,-0.2179,0.9025,0.371016]],
                  [-1.1669,-0.17989,0.85137],
                  [-2.99,-0.12,0.94,4.06,1.29,-4.12]],
              4:[[[2.025334948912725, 0.9435231432919353, 1.3268344716975315], [-0.08137147213675607, -0.16717652116184525, 0.10456194492167763, 0.9769838760024222]],[1.74589694978219,0.873373350569398,1.23301330552001],[0,0,0,0,0,0]],
              5:[]}

# for thetas in ik.SolveIKLimits(*test_cases[4][0]):
#   pprint(thetas)
#   pprint(ik.SolveFK(thetas))

for thetas in ik.SolveIK(*test_cases[4][0], debug = False):
  pprint(ik.SolveFK(thetas))


⎡2.02533494891272 ⎤
⎢                 ⎥
⎢0.943523143291935⎥
⎢                 ⎥
⎣1.32683447169753 ⎦
⎡2.02533494891273 ⎤
⎢                 ⎥
⎢0.943523143291935⎥
⎢                 ⎥
⎣1.32683447169753 ⎦


In [89]:
%timeit ik.SolveIKCheapest([0]*6, *test_cases[3][0])

34.6 ms ± 2.55 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Forward Kinematics Playground

In [16]:
T0_1,T1_2,T2_3,T3_4,T4_5,T5_6,T6_EE = ik.DHMatrices()
def e(q11, q22, q33, q44, q55, q66, joints):
  from kuka_arm.scripts.inverse_kinematics import q1,q2,q3,q4,q5,q6
  mat = reduce(lambda acc, M: acc * M, [T1_2,T2_3,T3_4,T4_5,T5_6,T6_EE][:(joints-1)], T0_1)
  return pprint(mat.evalf(subs={q1: q11, q2: q22, q3: q33, q4: q44, q5: q55, q6: q66}, prec = 5))

spec = (-3.14,3.14,0.01)
widgets.interact(e, q11=spec, q22=spec, q33=spec, q44=spec, q55=spec, q66=spec, joints=(2,7))

⎡2.5365e-6  -0.0015927     -1.0      -2.153 ⎤
⎢                                           ⎥
⎢0.0015927     -1.0     0.0015927   0.003429⎥
⎢                                           ⎥
⎢  -1.0     -0.0015927  1.7694e-16   1.946  ⎥
⎢                                           ⎥
⎣    0          0           0         1.0   ⎦


### R3_EE Matrix (symbols)

In [26]:
T3_EE = simplify(T3_4*T4_5*T5_6*T6_EE)
R3_EE = T3_EE[0:3, 0:3]
pprint(simplify(R3_EE))
#pprint(simplify(sqrt(R3_EE[0,2]**2 + R3_EE[2, 2]**2)))

⎡-sin(q₄)⋅sin(q₆) + cos(q₄)⋅cos(q₅)⋅cos(q₆)  -sin(q₄)⋅cos(q₆) - sin(q₆)⋅cos(q₄
⎢                                                                             
⎢             sin(q₅)⋅cos(q₆)                             -sin(q₅)⋅sin(q₆)    
⎢                                                                             
⎣-sin(q₄)⋅cos(q₅)⋅cos(q₆) - sin(q₆)⋅cos(q₄)  sin(q₄)⋅sin(q₆)⋅cos(q₅) - cos(q₄)

)⋅cos(q₅)  -sin(q₅)⋅cos(q₄)⎤
                           ⎥
               cos(q₅)     ⎥
                           ⎥
⋅cos(q₆)   sin(q₄)⋅sin(q₅) ⎦


## Symbolic computations (abandoned)

In [ ]:
def DHTransform(alpha, a, q, d):
  return Matrix([
    [cos(q),            -sin(q),           0,           a],
    [sin(q)*cos(alpha), cos(q)*cos(alpha), -sin(alpha), -sin(alpha)*d],
    [sin(q)*sin(alpha), cos(q)*sin(alpha), cos(alpha),  cos(alpha)*d ],
    [0,                 0,                 0,           1            ]])

q1, q2, q3, q4, q5, q6 = symbols('q1:7')
alpha = [0, -pi/2, 0, -pi/2, pi/2, -pi/2, 0]
a = symbols('a0:7')
d1, d2, d3, d4, d5, d6, d_ee = symbols('d1:7,d_ee')

constants = dict(
  list(zip(a, [0, 0.35, 1.25, -0.054, 0, 0, 0])) +
  list(zip([d1, d2, d3, d4, d5, d6, d_ee], [0.75, 0, 0, 1.5, 0, 0, 0.303])))

T0_1 = DHTransform(alpha[0], a[0], q1, d1)
T1_2 = DHTransform(alpha[1], a[1], q2 - pi/2, d2)
T2_3 = DHTransform(alpha[2], a[2], q3, d3)
T3_4 = DHTransform(alpha[3], a[3], q4, d4)
T4_5 = DHTransform(alpha[4], a[4], q5, d5)
T5_6 = DHTransform(alpha[5], a[5], q6, d6)
T6_EE = DHTransform(alpha[6], a[6], 0, d_ee)

# def e(q11, q22, q33, q44, q55, q66):
#   subs = {q1: q11, q2: q22, q3: q33, q4: q44, q5: q55, q6: q66}
#   subs.update(constants)
#   return pprint(mat.evalf(subs=subs, prec = 5))

# spec = (-3.14,3.14,0.01)
# widgets.interact(e, q11=spec, q22=spec, q33=spec, q44=spec, q55=spec, q66=spec)

In [ ]:
mat = reduce(lambda acc, M: simplify(acc * M), [T0_1,T1_2,T2_3,T3_4,T4_5,T5_6,T6_EE])
subs = {q1: 2.86, q2: 0, q3: 0, q4:0 , q5: 0, q6: 0}
subs.update(constants)
mat.evalf(subs=subs, prec = 5)

In [88]:
s = '''
0.3631; -0.012039; 3.4913
-0.69656; -0.098579; -0.38459; 0.59765
'''.strip().split('\n')
ee_pos_vals = list(map(float, s[0].split('; ')))
quaternion_vals = list(map(float, s[1].split('; ')))
ee_pos_vals, quaternion_vals

([0.3631, -0.012039, 3.4913], [-0.69656, -0.098579, -0.38459, 0.59765])